# 0.0 Entender o Problema

- Objetivo do problema:

    - 1.0 Previsão do primeiro destino que o um novo usuário irá escolher.

    - Porque?

        - Qual o tipo do modelo de negócio do Airbnb?

            - Marketplace (conectar oferta e demanda, e poder ganhar em cima dessa transação).

            - Oferta (Pessoas oferecem acomodação)

                - Tamanho do portifólio
                - Diversidade/densidade de portifólio.
                - Preço médio.

            - Demanda (Pessoas procurando acomodação)

                - Número de usuários
                - LTV (Lifetime Value - Quanto que a pessoa vai gastar com a empresa)
                - CAC (Client Acquisition Cost - Custo por cliente)

                Gross Revenue (Margem de lucro) = (fee * Número de usuário) - CAC
                **FEE => É a diferença entre o preço médio e o valor dado a pessoa que oferece a acomadação.**

- Proposta da Solução:

    - Modelo de predição do primeiro destino de um novo usuário. Tipo de entrega:

        - 1.0. Salvar a predição em tabelas no banco de dados para ser acessada por meio de uma ferramenta de visualização.
        - 2.0. Construção de API e acesso por meio dela:
        
            - Input: Usuário coloca suas características.
            - Output: Mostra as características do usuário e a **predição do destino**.

    - Ciclos para proposta de solução: 16 Ciclos.

# Ciclo 01 <font color = 'red'> - End to End </font>

## 0.0 Imports

In [53]:
import os
import pathlib

import pandas   as pd
import numpy    as np
#import matplotlib.pyplot as plt

from sklearn import model_selection as ms
from sklearn import preprocessing   as pp
from sklearn import metrics         as m
#from scikitplot import metrics      as mt
from tensorflow.keras import models as ml

from tensorflow.keras import layers as l

### 0.1. Helper Functions

In [54]:
# Diretório raíz
PACKAGE_ROOT = pathlib.Path('.').resolve().parent

# Caminhos para os diretórios dos dados
DATADIR = os.path.join(PACKAGE_ROOT, 'data')
DATADIR_RAW = os.path.join(DATADIR, 'raw')
DATADIR_INTERIM = os.path.join(DATADIR, 'interim')
DATADIR_PREPROCESSED = os.path.join(DATADIR, 'preprocessed')

# Files raw Data
FILE_TRAIN = os.path.join(DATADIR_RAW,'train_users_2.csv')
FILE_SESSIONS = os.path.join(DATADIR_RAW,'sessions.csv')
FILE_TEST = os.path.join(DATADIR_RAW,'test_users.csv')


### 0.2. Loading Data

In [55]:
df_raw = pd.read_csv(FILE_TRAIN, low_memory=False)
df_raw.shape

(213451, 16)

In [56]:
#b = r'D:\CDS\Projetos\PA01\airbnb_predict\Source\sessions.csv'
#df_sessions = pd.read_csv(b,low_memory=True)
#df_sessions.shape

In [57]:
#df_sessions.sample(2)

## 1.0. Data Description

In [58]:
df1 = df_raw.copy()

In [59]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of collumns: {}'.format(df1.shape[1]))

Number of rows: 213451
Number of collumns: 16


In [9]:
df1.head(2)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF


### 1.1 Data info:

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       213451 non-null  object 
 1   date_account_created     213451 non-null  object 
 2   timestamp_first_active   213451 non-null  int64  
 3   date_first_booking       88908 non-null   object 
 4   gender                   213451 non-null  object 
 5   age                      125461 non-null  float64
 6   signup_method            213451 non-null  object 
 7   signup_flow              213451 non-null  int64  
 8   language                 213451 non-null  object 
 9   affiliate_channel        213451 non-null  object 
 10  affiliate_provider       213451 non-null  object 
 11  first_affiliate_tracked  207386 non-null  object 
 12  signup_app               213451 non-null  object 
 13  first_device_type        213451 non-null  object 
 14  firs

Obs: A coluna date_account_created tem que ser Datetime.

### 1.2 NA check:

In [11]:
df1.isna().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

In [12]:
# Remove missing values
df1 = df1.dropna()
df1.shape

(68171, 16)

### 1.3 Change Data Type:

In [13]:
df1.dtypes

id                          object
date_account_created        object
timestamp_first_active       int64
date_first_booking          object
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
country_destination         object
dtype: object

In [14]:
# date_account_created
df1['date_account_created'] = pd.to_datetime(df1['date_account_created'])
# timestamp_first_active
df1['timestamp_first_active'] = pd.to_datetime(df1['timestamp_first_active'], format='%Y%m%d%H%M%S')

# date_first_booking
df1['date_first_booking'] = pd.to_datetime(df1['date_first_booking'])
# age
df1['age']=df1['age'].astype(np.int64)

In [15]:
df1.dtypes

id                                 object
date_account_created       datetime64[ns]
timestamp_first_active     datetime64[ns]
date_first_booking         datetime64[ns]
gender                             object
age                                 int64
signup_method                      object
signup_flow                         int64
language                           object
affiliate_channel                  object
affiliate_provider                 object
first_affiliate_tracked            object
signup_app                         object
first_device_type                  object
first_browser                      object
country_destination                object
dtype: object

### 1.4 Check balanced data:

In [16]:
# Observando o balanceamento da variável resposta
df1['country_destination'].value_counts(normalize=True)

# Obs: Ao excluir as linhas vazias, foi excluído todo uma classe. (NDF = na)

US       0.708864
other    0.110854
FR       0.054085
IT       0.029763
GB       0.025935
ES       0.024864
CA       0.015696
DE       0.012454
NL       0.008801
AU       0.006366
PT       0.002318
Name: country_destination, dtype: float64

In [17]:
df = df1.copy()

## 2.0. Data Filtering

### 2.1. Filteringo rows

### 2.2. Columns selection

## 3.0 Data Preparation

In [18]:
cols_categoric = ['gender',
                'signup_method',
                'language',
                'affiliate_channel',
                'affiliate_provider',
                'first_affiliate_tracked',
                'signup_app',
                'first_device_type',
                'first_browser']
# dummy variable
df_dummy = pd.get_dummies( df.drop( ['id','country_destination'], axis=1 ) ) # Retirando as variáveis do ID e a variável resposta (target)

#join id and country destination 
df = pd.concat( [df[['id','country_destination']],df_dummy], axis=1 )

In [19]:
df.head()

,id,country_destination,date_account_created,timestamp_first_active,date_first_booking,age,signup_flow,gender_-unknown-,gender_FEMALE,gender_MALE,...,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser
2,4ft3gnwmtx,US,2010-09-28,2009-06-09 23:12:47,2010-08-02,56,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,bjjt8pjhuk,other,2011-12-05,2009-10-31 06:01:29,2012-09-08,42,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,87mebub9p4,US,2010-09-14,2009-12-08 06:11:05,2010-02-18,41,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,lsw9q7uk0j,US,2010-01-02,2010-01-02 01:25:58,2010-01-05,46,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
7,0d01nltbrs,US,2010-01-03,2010-01-03 19:19:05,2010-01-13,47,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


## 4.0. Feature Selection

In [20]:
df.sample().T

,1288
id,i3b12vxe69
country_destination,US
date_account_created,2010-08-04 00:00:00
timestamp_first_active,2010-08-04 16:51:11
date_first_booking,2010-08-04 00:00:00
...,...
first_browser_Sogou Explorer,0
first_browser_Stainless,0
first_browser_TenFourFox,0
first_browser_TheWorld Browser,0


In [21]:
cols_drop = ['id','date_account_created','timestamp_first_active','date_first_booking'] # Original Dates
df=df.drop(cols_drop,axis=1)
#df.dtypes

In [22]:
X = df.drop('country_destination', axis=1)
y = df['country_destination'].copy()

In [23]:
X

,age,signup_flow,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,signup_method_google,language_ca,...,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser
2,56,3,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,41,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,46,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,47,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213415,55,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213439,43,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213441,34,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213443,36,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## 5.0. Machine Learning Model - Neural Network MLP

In [24]:
# Split Dataset into training and test
x_train, x_test, y_train, y_test = ms.train_test_split(
                                                        X,
                                                        y,
                                                        test_size = 0.25,
                                                        random_state=42
                                                        )

In [25]:
#y_train.values # Tem que transformar em valores numéricos. Utiliza-se o One Hot Encoding

In [26]:
#y_train.values.reshape(-1,1)

In [27]:
ohe = pp.OneHotEncoder()
y_ohe = y_train.values.reshape(-1,1)
y_train_nn = ohe.fit_transform(y_ohe).toarray()

In [28]:
# Model definition
model = ml.Sequential()
model.add( l.Dense (
                100,
                input_dim =x_train.shape[1],
                activation = 'relu' 
                ) ) # Irá adicionar as Layers da rede neural .. Camada de entrada

model.add( l.Dense (
                    11, #11 por causa do número de saídas
                    activation = 'softmax' 
                    )
                    )

# Model compile
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])



In [29]:
model.fit(x_train, y_train_nn, epochs=100)

Epoch 1/100
1598/1598 [==============================] - 4s 2ms/step - loss: 1.3547 - accuracy: 0.6985
Epoch 2/100
1598/1598 [==============================] - 2s 2ms/step - loss: 1.2441 - accuracy: 0.7066
Epoch 3/100
1598/1598 [==============================] - 3s 2ms/step - loss: 1.2237 - accuracy: 0.7063
Epoch 4/100
1598/1598 [==============================] - 2s 1ms/step - loss: 1.2068 - accuracy: 0.7067
Epoch 5/100
1598/1598 [==============================] - 2s 1ms/step - loss: 1.1881 - accuracy: 0.7068
Epoch 6/100
1598/1598 [==============================] - 2s 2ms/step - loss: 1.1757 - accuracy: 0.7070
Epoch 7/100
1598/1598 [==============================] - 2s 2ms/step - loss: 1.1702 - accuracy: 0.7070
Epoch 8/100
1598/1598 [==============================] - 2s 1ms/step - loss: 1.1600 - accuracy: 0.7071
Epoch 9/100
1598/1598 [==============================] - 2s 1ms/step - loss: 1.1514 - accuracy: 0.7071
Epoch 10/100
1598/1598 [==============================] - 3s 2ms/step - l

## 6.0. NN Performance

In [30]:
# prediction 
pred_nn = model.predict( x_test )

In [31]:
# Invert prediction
yhat_nn = ohe.inverse_transform( pred_nn )

In [46]:
classes = ohe.categories_[0].tolist()


['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NL', 'PT', 'US', 'other']

In [32]:


# Prediction prepare
y_test_nn = y_test.to_numpy()
yhat_nn = yhat_nn.reshape(1 , -1 )[0]

In [60]:
# Accuracy
m.accuracy_score( 
                y_test_nn, 
                yhat_nn
                )
# Confusion Matrix


#mt.plot_confusion_matrix(
#                        y_test_nn,
#                        yhat_nn, 
#                        normalize=False, 
#                        figsize=(12,12)
#                        )

0.7098515519568152

In [62]:
cr = m.classification_report(y_test_nn, yhat_nn, target_names=classes)
print(cr)

C:\Users\kayss\AppData\Local\pypoetry\Cache\virtualenvs\airbnb-pred-O86c4xb--py3.9\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kayss\AppData\Local\pypoetry\Cache\virtualenvs\airbnb-pred-O86c4xb--py3.9\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AU       0.00      0.00      0.00       110
          CA       0.00      0.00      0.00       272
          DE       0.00      0.00      0.00       218
          ES       0.00      0.00      0.00       436
          FR       0.18      0.00      0.00       917
          GB       0.00      0.00      0.00       439
          IT       0.00      0.00      0.00       503
          NL       0.00      0.00      0.00       130
          PT       0.00      0.00      0.00        37
          US       0.71      1.00      0.83     12126
       other       0.17      0.00      0.00      1855

    accuracy                           0.71     17043
   macro avg       0.10      0.09      0.08     17043
weighted avg       0.53      0.71      0.59     17043



C:\Users\kayss\AppData\Local\pypoetry\Cache\virtualenvs\airbnb-pred-O86c4xb--py3.9\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
cm = m.confusion_matrix(y_test_nn, yhat_nn)

In [63]:
disp = m.ConfusionMatrixDisplay(confusion_matrix=cm,
                        display_labels=classes)
disp.plot()


ImportError: ConfusionMatrixDisplay.plot requires matplotlib. You can install matplotlib with `pip install matplotlib`